## Create Wav2Vec2CTCTokenizer

In [ ]:
!pip install datasets
!pip install transformers # transformers는 최신 버전 회소 4.19.4

In [1]:
from datasets import load_dataset
import pandas as pd
import numpy as np

In [2]:
# huggingface dataset
all_data = load_dataset('csv',data_files='./order_speech_ko_test.csv',split='train')

Using custom data configuration default-5004c11ebe3a2984
Reusing dataset csv (C:\Users\sangt\.cache\huggingface\datasets\csv\default-5004c11ebe3a2984\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


## add audio array

In [3]:
import librosa

In [4]:
def load_audio(batch):
#     batch['array'],_ = librosa.load('./dataset/audio/'+batch['src'],sr=16000)
    batch['array'],_ = librosa.load(batch['src'], sr=16000)
    return batch

In [5]:
all_data = all_data.map(load_audio)

Loading cached processed dataset at C:\Users\sangt\.cache\huggingface\datasets\csv\default-5004c11ebe3a2984\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-ef8f806719d74c07.arrow


In [6]:
# 데이터 확인
all_data[0]['array'][1*16000:2*16000]

[-0.0015869140625,
 -0.001007080078125,
 -0.001129150390625,
 -0.00128173828125,
 -0.0009765625,
 -0.00042724609375,
 -0.000274658203125,
 -0.00164794921875,
 -0.00250244140625,
 -0.001861572265625,
 -0.00091552734375,
 -0.000640869140625,
 -0.000396728515625,
 0.00048828125,
 0.00103759765625,
 0.00042724609375,
 0.000274658203125,
 0.0001220703125,
 0.000335693359375,
 0.000732421875,
 0.000274658203125,
 9.1552734375e-05,
 0.000152587890625,
 9.1552734375e-05,
 3.0517578125e-05,
 0.00048828125,
 0.0003662109375,
 0.000335693359375,
 0.0015869140625,
 0.0018310546875,
 0.001739501953125,
 0.00146484375,
 0.001312255859375,
 0.002044677734375,
 0.00152587890625,
 0.001190185546875,
 0.0010986328125,
 0.0003662109375,
 -6.103515625e-05,
 0.0001220703125,
 -0.000274658203125,
 -0.00115966796875,
 -0.000732421875,
 -0.00128173828125,
 -0.001434326171875,
 -0.0018310546875,
 -0.00238037109375,
 -0.001007080078125,
 0.000335693359375,
 0.00042724609375,
 0.000640869140625,
 0.0006408691406

In [7]:
print(all_data.shape)
all_data[0]

(301, 5)


{'Unnamed: 0': 0,
 'src': './dataset/KlecSpeech/Validation/D01/E01/S000028/000000.wav',
 'text': '여러분 안녕하세요',
 'text_len': 9,
 'array': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.

In [8]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_sepcial_characters(batch):
    try:
        debug = batch["text"]
        batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    except TypeError as e:
        print(debug, "Not a valid choice! Try again : ", e)
        
    return batch

In [9]:
#TPU나 GPU없으면 1시간 정도 걸림 32971건기준
# 메모리 부족으로 나눠서 진행 필요. (16GB 이상)
remove_spectial_char_data = all_data.map(remove_sepcial_characters)

Loading cached processed dataset at C:\Users\sangt\.cache\huggingface\datasets\csv\default-5004c11ebe3a2984\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-f90d273093fcc226.arrow


In [10]:
remove_spectial_char_data

Dataset({
    features: ['Unnamed: 0', 'src', 'text', 'text_len', 'array'],
    num_rows: 301
})

In [11]:
# 여기서 시간이 무진장 걸림 1000건 이상은 너무 오래 걸림(!시간 이상)
df = pd.DataFrame(remove_spectial_char_data)
df.head()

,Unnamed: 0,src,text,text_len,array
0,0,./dataset/KlecSpeech/Validation/D01/E01/S00002...,여러분 안녕하세요,9,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,./dataset/KlecSpeech/Validation/D01/E01/S00002...,친구들이 독해의 열매를 맺어 독해력이 쑥쑥 자랄 수 있도록 도와주는 혜은 선생님 이...,83,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,./dataset/KlecSpeech/Validation/D01/E01/S00002...,오늘의 생각 씨앗은요 히읗 비읍 이에요 선생님은 어렸을 때 이 생각 씨앗을 입고 싶...,120,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,./dataset/KlecSpeech/Validation/D01/E01/S00002...,색동저고리 함께 읽어보도록 하겠습니다,20,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,./dataset/KlecSpeech/Validation/D01/E01/S00002...,와 엄마 너무 예뻐요,11,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [12]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}


In [13]:
char_vocab = remove_spectial_char_data.map(
    extract_all_chars,
    batched=True,
    batch_size=-1,
    keep_in_memory=True,
    remove_columns=remove_spectial_char_data.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [14]:
char_vocab

Dataset({
    features: ['vocab', 'all_text'],
    num_rows: 1
})

In [15]:
vocab_list = list(set(char_vocab["vocab"][0]))

In [16]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'덩': 0,
 '더': 1,
 '우': 2,
 '렸': 3,
 '워': 4,
 '징': 5,
 '같': 6,
 '웃': 7,
 '이': 8,
 '품': 9,
 '급': 10,
 '몽': 11,
 '슨': 12,
 '됐': 13,
 '깔': 14,
 '식': 15,
 '뭔': 16,
 '련': 17,
 '들': 18,
 '오': 19,
 '앗': 20,
 '의': 21,
 '빔': 22,
 '해': 23,
 '꿈': 24,
 '록': 25,
 '게': 26,
 '석': 27,
 '마': 28,
 '직': 29,
 '다': 30,
 '릇': 31,
 '변': 32,
 '모': 33,
 '잃': 34,
 '커': 35,
 '끔': 36,
 '실': 37,
 '답': 38,
 '말': 39,
 '영': 40,
 '꿔': 41,
 '숲': 42,
 '쁜': 43,
 '투': 44,
 '끌': 45,
 '형': 46,
 '느': 47,
 '동': 48,
 '덕': 49,
 '맙': 50,
 '참': 51,
 '약': 52,
 '채': 53,
 '집': 54,
 '스': 55,
 '줄': 56,
 '몰': 57,
 '짓': 58,
 '서': 59,
 '르': 60,
 '았': 61,
 '네': 62,
 '윷': 63,
 '색': 64,
 '금': 65,
 '길': 66,
 '독': 67,
 '늑': 68,
 '닐': 69,
 '양': 70,
 '름': 71,
 '완': 72,
 '근': 73,
 '속': 74,
 '잘': 75,
 '학': 76,
 '벌': 77,
 '데': 78,
 '재': 79,
 '믿': 80,
 '떻': 81,
 '조': 82,
 '가': 83,
 '불': 84,
 '칸': 85,
 '열': 86,
 '롭': 87,
 '히': 88,
 '소': 89,
 '넌': 90,
 '질': 91,
 '난': 92,
 '욕': 93,
 '힘': 94,
 '으': 95,
 '손': 96,
 '필': 97,
 '람': 98,
 '보': 99,
 '력': 100,

In [17]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

494

In [18]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [19]:
!pip install transformers

In [21]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json",
                                 unk_token="[UNK]",
                                 pad_token="[PAD]",
                                 word_delimiter_token="|")

## Create XLSR-Wav2Vec2 Feature Extractor

In [22]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True,
                                             return_attention_mask=True)

In [23]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor,
                              tokenizer=tokenizer)

In [24]:
processor

Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

- tokenizer: PreTrainedTokenizer(name_or_path='', vocab_size=494, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'})

In [25]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [26]:
# processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-ready")

## Preprocess Data

In [27]:
def prepare_dataset(batch):
    # batched output is "un-batched"
    batch["input_values"] = processor(batch["array"], sampling_rate=16000).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [28]:
order_voice = remove_spectial_char_data.map(
    prepare_dataset,
    remove_columns=remove_spectial_char_data.column_names,
    # num_proc=4 num_proc=4이 옵션은 주피터 랩에서는 안된다. 하드웨어와 연관이 있는 듯? colab에서는 이 옵션을 추가해도 된다.
#     # 병렬 처리에 관련된 옵션이라고 한다.
)

  0%|          | 0/301 [00:00<?, ?ex/s]

# Training

## Set-up Trainer

In [29]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [30]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [31]:
pip install jiwer

  Using cached jiwer-2.3.0-py3-none-any.whl (15 kB)
  Using cached python-Levenshtein-0.12.2.tar.gz (50 kB)
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-win_amd64.whl size=80836 sha256=d42acc2d29eda69da12694f848bc370b10f9995854a1208fc78b004b2d12766c
  Stored in directory: c:\users\sangt\appdata\local\pip\cache\wheels\05\5f\ca\7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein
Note: you may need to restart the kernel to use updated packages.


In [32]:
from datasets import load_dataset, load_metric, Audio

wer_metric = load_metric("wer")

In [33]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

## Import Model

In [34]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.bias', 'project_hid.weight', 'project_q.weight', 'quantizer.weight_proj.weight', 'project_hid.bias', 'project_q.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

XLSR-Wav2Vec2의 첫 번째 구성 요소는 원시 음성 신호에서 음향적으로 의미가 있지만 문맥적으로 독립적인 기능을 추출하는 데 사용되는 CNN 계층 스택으로 구성됩니다.  
모델의 이 부분은 사전 교육 중에 이미 충분히 훈련되었으며 논문에 명시된 바와 같이 더 이상 미세 조정할 필요가 없습니다. 따라서 특징 추출 부분의 모든 파라미터에 대해 require_grad를 False로 설정할 수 있다.

In [35]:
model.freeze_feature_extractor()

C:\Users\sangt\anaconda3\envs\STT\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:1677: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


메모리를 절약하기 위해 그라데이션 체크포인팅을 활성화

In [36]:
model.gradient_checkpointing_enable()

## TrainingArguments

In [37]:
# cuda 장비 없으면 에러 발생 

from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
  output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=3,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

ValueError: Mixed precision training with AMP or APEX (`--fp16` or `--bf16`) and half precision evaluation (`--fp16_full_eval` or `--bf16_full_eval`) can only be used on CUDA devices.

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=order_voice, # train_ds
    tokenizer=processor.feature_extractor,
)

## Model Training

In [ ]:
data_collator(remove_spectial_char_data)

In [ ]:
trainer.train()

CTC 손실을 사용하여 더 큰 데이터 세트에서 더 큰 모델을 미세 조정하려면 [여기서](https://github.com/huggingface/transformers/tree/master/examples/pytorch/speech-recognition#connectionist-temporal-classification-without-language-model-ctc-wo-lm) 공식 음성 인식 예를 살펴봐야 한다.

## Model predict

In [ ]:
model(audio_data)